## Redes Neuronales Artificiales – Propagación hacia Adelante

En este laboratorio, construiremos una red neuronal desde cero y programaremos cómo realiza predicciones utilizando la propagación hacia adelante (forward propagation).

Cabe destacar que todas las bibliotecas de aprendizaje profundo ya tienen implementados los procesos completos de entrenamiento y predicción, por lo que en la práctica no sería necesario construir una red neuronal desde cero. Sin embargo, realizar este laboratorio te ayudará a comprender mejor cómo funcionan las redes neuronales y cómo operan internamente.

## Objetivo de este cuaderno
* Construir una red neuronal
* Calcular la suma ponderada en cada nodo
* Calcular la activación de cada nodo
* Utilizar la propagación hacia adelante (*forward propagation*) para propagar los datos


Repasemos cómo una red neuronal realiza predicciones mediante el proceso de propagación hacia adelante (forward propagation). A continuación se muestra una red neuronal que recibe dos entradas, tiene una capa oculta con dos nodos y una capa de salida con un solo nodo.

<img src="http://cocl.us/neural_network_example" alt="Neural Network Example" width="600px">

Comencemos inicializando aleatoriamente los pesos y los sesgos (biases) de la red. Tenemos 6 pesos y 3 sesgos, uno para cada nodo de la capa oculta, así como uno para el nodo de la capa de salida.

In [1]:
import numpy as np # import Numpy library to generate

weights = np.around(np.random.uniform(size=6), decimals=2) # initialize the weights
biases = np.around(np.random.uniform(size=3), decimals=2) # initialize the biases

In [2]:
print(weights)
print(biases)

[0.69 0.39 0.44 0.45 0.45 0.52]
[0.58 0.22 0.29]


Ahora que ya tenemos definidos los pesos y los sesgos de la red, calculemos la salida para una entrada dada, 𝑥1 y 𝑥2

In [7]:
x_1 = 0.5 # input 1
x_2 = 0.85 # input 2

print('x1: {} , x2: {}'.format(x_1, x_2))

x1: 0.5 , x2: 0.85


Comencemos calculando la suma ponderada de las entradas, 𝑧1,1 en el primer nodo de la capa oculta.

In [8]:
z_11 = x_1 * weights[0] + x_2 * weights[1] + biases[0]

print('La suma ponerada de las entradas en el primer nodo de la capa oculta es {}'.format(z_11))

La suma ponerada de las entradas en el primer nodo de la capa oculta es 1.2565


A continuación, calculemos la suma ponderada de las entradas,
𝑧1,2, en el segundo nodo de la capa oculta. Asigna este valor a z_12

In [9]:
z_12 = x_1 * weights[2] + x_2 * weights[3] + biases[1]

print('La suma ponerada de las entradas en el segundo nodo de la capa oculta es {}'.format(z_11))

La suma ponerada de las entradas en el segundo nodo de la capa oculta es 1.2565


A continuación, suponiendo una función de activación sigmoide, calculemos la activación del primer nodo, 𝑎1,1 en la capa oculta.

In [10]:
a_11 = 1.0 / (1.0 + np.exp(-z_11))

print('La activación del primer nodo in la capa oclta es {}'.format(np.around(a_11, decimals=4)))

La activación del primer nodo in la capa oclta es 0.7784


In [12]:
a_12 = 1.0 / (1.0 + np.exp(-z_12))

print('La activación del segundo nodo in la capa oclta es {}'.format(np.around(a_12, decimals=4)))

La activación del segundo nodo in la capa oclta es 0.6948


Ahora, estas activaciones servirán como entradas para la capa de salida. Por lo tanto, calculemos la suma ponderada de estas entradas en el nodo de la capa de salida. Asigna este valor a z_2.

In [14]:
z_2=a_11*weights[4]+a_12*weights[5]+biases[2]
print(z_2)

1.0015690729539006


Finalmente, calculemos la salida de la red como la activación del nodo en la capa de salida. Asigna este valor a a_2

In [16]:
a_2=1.0 / (1.0 + np.exp(-z_2))

In [19]:
print('La slida para la red neuronal para x1 = 0.5 y x2 = 0.85 es {}'.format(np.around(a_2, decimals=4)))

La slida para la red neuronal para x1 = 0.5 y x2 = 0.85 es 0.7314


Obviamente, las redes neuronales para problemas reales están compuestas por muchas capas ocultas y muchos más nodos en cada capa. Por lo tanto, no podemos seguir realizando predicciones utilizando este enfoque tan ineficiente de calcular manualmente la suma ponderada y la activación en cada nodo.

Para poder programar una forma automática de realizar predicciones, generalicemos nuestra red.

Una red general recibiría n entradas, tendría múltiples capas ocultas, donde cada capa oculta tendría m nodos, y contaría con una capa de salida.

Aunque la red que se muestra tiene una sola capa oculta, nosotros programaremos la red para que pueda tener varias capas ocultas. De manera similar, aunque la red muestra una capa de salida con un solo nodo, programaremos la red para que pueda tener más de un nodo en la capa de salida.

<img src="http://cocl.us/general_neural_network" alt="Neural Network General" width="600px">


## Construye una red neuronal

Comencemos por definir formalmente la estructura de la red

In [20]:
n = 2 # number of inputs
num_hidden_layers = 2 # number of hidden layers
m = [2, 2] # number of nodes in each hidden layer
num_nodes_output = 1 # number of nodes in the output layer

Ahora que hemos definido la estructura de la red, procedamos a inicializar los pesos y los sesgos de la red con números aleatorios. Para poder inicializar los pesos y los sesgos con valores aleatorios, necesitaremos importar la librería NumPy

In [21]:
import numpy as np # import the Numpy library

num_nodes_previous = n # number of nodes in the previous layer

network = {} # initialize network an an empty dictionary

# loop through each layer and randomly initialize the weights and biases associated with each node
# notice how we are adding 1 to the number of hidden layers in order to include the output layer
for layer in range(num_hidden_layers + 1):

    # determine name of layer
    if layer == num_hidden_layers:
        layer_name = 'output'
        num_nodes = num_nodes_output
    else:
        layer_name = 'layer_{}'.format(layer + 1)
        num_nodes = m[layer]

    # initialize weights and biases associated with each node in the current layer
    network[layer_name] = {}
    for node in range(num_nodes):
        node_name = 'node_{}'.format(node+1)
        network[layer_name][node_name] = {
            'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
            'bias': np.around(np.random.uniform(size=1), decimals=2),
        }

    num_nodes_previous = num_nodes

print(network) # print network

{'layer_1': {'node_1': {'weights': array([0.07, 0.56]), 'bias': array([1.])}, 'node_2': {'weights': array([0.81, 0.87]), 'bias': array([0.52])}}, 'layer_2': {'node_1': {'weights': array([0.5 , 0.73]), 'bias': array([0.41])}, 'node_2': {'weights': array([0.63, 0.26]), 'bias': array([0.51])}}, 'output': {'node_1': {'weights': array([0.37, 0.51]), 'bias': array([0.18])}}}


Entonces, con el código anterior, somos capaces de inicializar los pesos y los sesgos correspondientes a cualquier red, sin importar el número de capas ocultas ni el número de nodos en cada capa.

Sin embargo, pongamos todo este código dentro de una función para que podamos ejecutarlo de manera repetida cada vez que queramos construir una red neuronal.

In [22]:
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):

    num_nodes_previous = num_inputs # number of nodes in the previous layer

    network = {}

    # loop through each layer and randomly initialize the weights and biases associated with each layer
    for layer in range(num_hidden_layers + 1):

        if layer == num_hidden_layers:
            layer_name = 'output' # name last layer in the network output
            num_nodes = num_nodes_output
        else:
            layer_name = 'layer_{}'.format(layer + 1) # otherwise give the layer a number
            num_nodes = num_nodes_hidden[layer]

        # initialize weights and bias for each node
        network[layer_name] = {}
        for node in range(num_nodes):
            node_name = 'node_{}'.format(node+1)
            network[layer_name][node_name] = {
                'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
                'bias': np.around(np.random.uniform(size=1), decimals=2),
            }

        num_nodes_previous = num_nodes

    return network # return the network

#### Usa la función *initialize_network* para crear una red que:

1. reciba 5 entradas
2. tenga tres capas ocultas
3. tenga 3 nodos en la primera capa, 2 nodos en la segunda capa y 3 nodos en la tercera capa
4. tenga 1 nodo en la capa de salida

Llama a la red **small_network**.


In [24]:
small_network=initialize_network(5,3,[3,2,3],1)

## Calcular la suma ponderada en cada nodo

La suma ponderada en cada nodo se calcula como el producto punto entre las entradas y los pesos, más el sesgo. Por lo tanto, creemos una función llamada compute_weighted_sum que realice exactamente este cálculo.

In [25]:
def compute_weighted_sum(inputs, weights, bias):
    return np.sum(inputs * weights) + bias

Generemos 5 entradas para alimentar a **small_network**.

In [27]:
from random import seed
import numpy as np

np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('Las entradas para la red nueronal son {}'.format(inputs))

Las entradas para la red nueronal son [0.15 0.74 0.26 0.53 0.01]


#### Usa la función *compute_weighted_sum* para calcular la suma ponderada en el primer nodo de la primera capa oculta.


In [28]:
node_weights = small_network['layer_1']['node_1']['weights']
node_bias = small_network['layer_1']['node_1']['bias']

weighted_sum = compute_weighted_sum(inputs, node_weights, node_bias)
print('The weighted sum at the first node in the hidden layer is {}'.format(np.around(weighted_sum[0], decimals=4)))

The weighted sum at the first node in the hidden layer is 1.413


## Calcular la activación del nodo


Recordemos que la salida de cada nodo es simplemente una transformación no lineal de la suma ponderada. Para realizar esta transformación utilizamos funciones de activación. En este caso, usaremos la función sigmoide como función de activación. Por lo tanto, definamos una función que tome como entrada una suma ponderada y devuelva la transformación no lineal de dicha entrada utilizando la función sigmoide.

In [29]:
def node_activation(weighted_sum):
    return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

#### Usa la función *node_activation* para calcular la salida del primer nodo de la primera capa oculta.


In [30]:
node_output  = node_activation(compute_weighted_sum(inputs, node_weights, node_bias))
print('The output of the first node in the hidden layer is {}'.format(np.around(node_output[0], decimals=4)))

The output of the first node in the hidden layer is 0.8042


## Propagación hacia adelante


La última pieza para construir una red neuronal capaz de realizar predicciones es integrar todos los componentes. Por lo tanto, creemos una función que aplique las funciones compute_weighted_sum y node_activation a cada nodo de la red, y que propague los datos a lo largo de toda la red hasta la capa de salida, generando una predicción para cada nodo de la capa de salida.

La forma en que lograremos esto es mediante el siguiente procedimiento:

1. Comenzar con la capa de entrada como entrada para la primera capa oculta.
2. Calcular la suma ponderada en los nodos de la capa actual.
3. Calcular la salida de los nodos de la capa actual.
4. Establecer la salida de la capa actual como la entrada de la siguiente capa.
5. Avanzar a la siguiente capa de la red.
6. Repetir los pasos 2 a 5 hasta calcular la salida de la capa de salida.

In [31]:
def forward_propagate(network, inputs):

    layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer

    for layer in network:

        layer_data = network[layer]

        layer_outputs = []
        for layer_node in layer_data:

            node_data = layer_data[layer_node]

            # compute the weighted sum and the output of each node at the same time
            node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
            layer_outputs.append(np.around(node_output[0], decimals=4))

        if layer != 'output':
            print('The outputs of the nodes in hidden layer number {} is {}'.format(layer.split('_')[1], layer_outputs))

        layer_inputs = layer_outputs # set the output of this layer to be the input to next layer

    network_predictions = layer_outputs
    return network_predictions

#### Usa la función *forward_propagate* para calcular la predicción de nuestra red pequeña.


In [37]:
predictions = forward_propagate(small_network, inputs)
print('The predicted value by the network for the given input is {}'.format(np.around(predictions[0], decimals=4)))

The outputs of the nodes in hidden layer number 1 is [np.float64(0.8042), np.float64(0.7779), np.float64(0.6717)]
The outputs of the nodes in hidden layer number 2 is [np.float64(0.7694), np.float64(0.9143)]
The outputs of the nodes in hidden layer number 3 is [np.float64(0.6595), np.float64(0.7599), np.float64(0.714)]
The predicted value by the network for the given input is 0.9172


De esta manera, construimos el código para definir una red neuronal. Podemos especificar el número de entradas que puede recibir la red, el número de capas ocultas, así como el número de nodos en cada capa oculta, y el número de nodos en la capa de salida.

Primero utilizamos la función initialize_network para crear nuestra red neuronal y definir sus pesos y sesgos.

In [32]:
my_network = initialize_network(5, 3, [2, 3, 2], 3)

Luego, para una entrada dada,

In [ ]:
inputs = np.around(np.random.uniform(size=5), decimals=2)

Calculamos las predicciones de la red

In [36]:
predictions = forward_propagate(my_network, inputs)
print('The predicted values by the network for the given input are {}'.format(predictions))

The outputs of the nodes in hidden layer number 1 is [np.float64(0.8323), np.float64(0.8268)]
The outputs of the nodes in hidden layer number 2 is [np.float64(0.7693), np.float64(0.6876), np.float64(0.7362)]
The outputs of the nodes in hidden layer number 3 is [np.float64(0.8663), np.float64(0.8796)]
The predicted values by the network for the given input are [np.float64(0.895), np.float64(0.822), np.float64(0.8032)]
